# Packages

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math

from Assets.Loading_Datasets import *

# Dataset

In [0]:
train_set = []
for n in range(len(train_set_features)):
    image = np.zeros((102, 1))
    for i in range(102):
        image[i, 0] = train_set_features[n][i]

    label_value = int(train_set_labels[n])
    label = np.zeros((4, 1))
    label[label_value, 0] = 1

    train_set.append((image, label))

# Network Class

In [ ]:
class NeuralNetwork:

    def __init__(self, layer_sizes, epoch_number, batch_size, learning_rate, method, use_momentum=False):
        self.layer_sizes = layer_sizes
        self.network_size = len(layer_sizes)
        self.epoch_number = epoch_number
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.method = method
        self.use_momentum = use_momentum
        self.weights = {}
        self.biases = {}
        self.costs = []

        self.momentum = 0.9
        
        for layer_number in range(1, self.network_size):
            self.weights[layer_number] = np.random.randn(layer_sizes[layer_number], layer_sizes[layer_number - 1])
            self.biases[layer_number] = np.zeros((layer_sizes[layer_number], 1))

    def sigmoid(self, array, derive=False):
        if derive:
            return array * (1 - array)
        return 1 / (1 + np.exp(-array))

    def relu(self, array, derive=False):
        if derive:
            dx = np.ones_like(array)
            dx[array <= 0] = 0
            return dx
        return np.maximum(0, array)

    def softmax(self, array):
        return np.exp(array) / sum(np.exp(array))

    def train(self, data):
        g_weights = {}
        g_biases = {}
        g_acts = {}
        prev_g_weights = {}
        prev_g_biases = {}
        for layer_number in range(1, self.network_size):
            prev_g_weights[layer_number] = np.zeros(
                (self.layer_sizes[layer_number], self.layer_sizes[layer_number - 1]))
            prev_g_biases[layer_number] = np.zeros((self.layer_sizes[layer_number], 1))
        costs = []
        batch_cost = 0
        number_of_batches = math.ceil(len(data) / self.batch_size)
        print("Training ...")
        for epoch_number in range(self.epoch_number):
            np.random.shuffle(data)
            for batch_number in range(number_of_batches):
                for layer_number in range(1, self.network_size):
                    g_weights[layer_number] = np.zeros(
                        (self.layer_sizes[layer_number], self.layer_sizes[layer_number - 1]))
                    g_biases[layer_number] = np.zeros((self.layer_sizes[layer_number], 1))
                for i in range(self.batch_size):
                    image, label = data[self.batch_size * batch_number + i]
                    layers = {}
                    for layer_number in range(1, self.network_size):
                        layers[0] = image
                        if layer_number == self.network_size - 1:
                            layers[layer_number] = self.sigmoid(
                                np.matmul(self.weights[layer_number], layers[layer_number - 1]) + self.biases[
                                    layer_number])
                        else:
                            layers[layer_number] = self.sigmoid(
                                np.matmul(self.weights[layer_number], layers[layer_number - 1]) + self.biases[
                                    layer_number])
                    batch_cost += ((layers[self.network_size - 1] - label) ** 2).mean(axis=None)
                    g_acts[self.network_size - 1] = 2 * (layers[self.network_size - 1] - label)
                    for layer_number in range(self.network_size - 1, 0, -1):
                        if layer_number == self.network_size - 1:
                            act_deriv = self.sigmoid(layers[layer_number], True)
                        else:
                            act_deriv = self.sigmoid(layers[layer_number], True)
                        tmp = np.multiply(act_deriv, g_acts[layer_number])
                        g_biases[layer_number] += tmp
                        g_weights[layer_number] += np.matmul(tmp, layers[layer_number - 1].transpose())
                        if layer_number > 1:
                            g_acts[layer_number - 1] = np.matmul(self.weights[layer_number].transpose(), tmp)
                for layer_number in range(1, self.network_size):
                    if self.use_momentum:
                        gw = self.momentum * prev_g_weights[layer_number] - (
                                1.0 - self.momentum) * self.learning_rate * g_weights[
                                 layer_number] / self.batch_size
                        gb = self.momentum * prev_g_biases[layer_number] - (
                                1.0 - self.momentum) * self.learning_rate * g_biases[layer_number] / self.batch_size
                        prev_g_weights[layer_number] = gw
                        prev_g_biases[layer_number] = gb
                    else:
                        gw = self.learning_rate * g_weights[layer_number] / self.batch_size
                        gb = self.learning_rate * g_biases[layer_number] / self.batch_size

                    self.weights[layer_number] += gw
                    self.biases[layer_number] += gb

            costs.append(batch_cost / self.batch_size)
            batch_cost = 0
        print("Trained!")
        plt.plot(costs)
        plt.show()

    def test(self, data):
        corrects = 0
        for i in range(len(data)):
            image, label = data[i]
            layers = {}
            for layer_number in range(1, self.network_size):
                layers[0] = image
                if layer_number == self.network_size - 1:
                    layers[layer_number] = self.sigmoid(
                        np.matmul(self.weights[layer_number], layers[layer_number - 1]) + self.biases[layer_number])
                else:
                    layers[layer_number] = self.sigmoid(
                        np.matmul(self.weights[layer_number], layers[layer_number - 1]) + self.biases[layer_number])
            if layers[self.network_size - 1].argmax() == label.argmax():
                corrects += 1
        print(f"Accuracy: {round(100 * corrects / len(data), 3)}%")


Create neural network and train the data

In [0]:
print("Loading Data ...")
nn = NeuralNetwork([102, 150, 60, 4], epoch_number=50, batch_size=10, learning_rate=0.1, method="sigmoid", use_momentum=True)
nn.train(data=train_set[:400])

test the trained data

In [0]:
nn.test(train_set[:1600])

get test data

In [0]:
test_set = []
for n in range(len(test_set_features)):
    image2 = np.zeros((102, 1))
    for i in range(102):
        image2[i, 0] = test_set_features[n][i]

    label_value2 = int(test_set_labels[n])
    label2 = np.zeros((4, 1))
    label2[label_value2, 0] = 1

    test_set.append((image2, label2))

In [0]:
nn.test(test_set)